<a href="https://colab.research.google.com/github/franciscogarate/matact4fbs/blob/main/notebooks/9_Mejor_Estimacion_Decesos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cálculo de la Mejor Estimación de una póliza de Decesos (Vida Entera)

En este ejemplo, calcularemos la mejor estimación de una poliza de decesos (seguro vida entera) a prima única (sin entradas de primas).
El primer paso será descargarnos la base de datos clonando el repositorio de github:

In [ ]:
!git clone https://github.com/franciscogarate/matact4fbs/

Importamos las librerias necesarias, así como la tabla de mortalidad y curva libre de riesgo

In [ ]:
import pandas as pd
import numpy as np
from matact4fbs import lx, clr

Establezco las **hipótesis** de mi proyección:

In [ ]:
lx = lx.PASEM2020_Decesos_M_2orden.values.copy() # Tabla PASEM2020 Decesos 2orden Hombres
w = len(lx)                                      # Fin de tabla
edad = 50                                        # Edad en t=0
capital = 5000.                                  # Capital inicial
tasa_incr_capital = 0                            # Incremento del capital
tasa_caida = 0.03                                # Caida de cartera
tasa_gastos = 0.015                              # Gastos por poliza sobre el capital

Creo un **df** con **pd.date_range()** para realizar mis proyecciones. Para simplificar, calculamos con flujos anuales.

In [ ]:
df = pd.DataFrame(pd.date_range(start='2024-12-31',periods=(w - edad),freq='YE'), columns=['Fecha'])
df['t'] = df.index
df['edad'] = edad + df.index
df['lx'] = df['edad'].apply(lambda x: lx[x])
df.head()

Calculo la qx como diferencia entre lx y lx+1 dividida por el número de supervivientes al inicio de la proyección:

In [ ]:
df['qx'] = df['lx'].diff(-1).fillna(0)/df['lx'][0]
df['sum_qx'] = df['qx'].cumsum()
df['px'] = 1 - df['sum_qx']
df.head()

Aplicamos nuestro incremento de capital

In [ ]:
def incr_capital(capital, t):
  return capital * (1 + tasa_incr_capital) ** t

df['capital'] = incr_capital(capital, df.t)
df.head()

Calculamos los pagos probabilizados en casa periodo de la proyección. Sin incremento del capital, la suma de todos los períodos debería ser igual al capital inicial:

In [ ]:
df['pagos'] = df['capital'] * df['qx']
df.head()

In [ ]:
df.pagos.sum()

Aplicamos nuestra tasa de caída de la cartera:

In [ ]:
df['tasa_caida'] = tasa_caida
df['polizas_benef'] = (1 - df['tasa_caida']).cumprod()
df.head()

En este paso, calculamos los pagos por siniestralidad probabilizados (por fallecimiento y caída de cartera)



In [ ]:
df['pagos_prob'] = df['pagos'] * df['polizas_benef']
df.head(2)

Incluimos la curva libre de riesgo para calcular el factor de descuento:

In [ ]:
df['clr'] = df['t'].apply(lambda t: clr[t])
df['factor_desc'] = df.apply(lambda x: 1 / (1 + x.clr) ** (x.t), axis=1)
df.head(2)

Incluimos los gastos

In [ ]:
df['gastos'] = df['capital'] * tasa_gastos
df['gastos_prob'] = df['gastos'] * df['px'] * df['polizas_benef']
df.head(2)

Calculamos las salidas totales (siniestralidad + gastos)

In [ ]:
df['salidas_prob'] = df['pagos_prob'] + df['gastos_prob']
df.head(2)

Por último, descontamos los flujos probabilizados utilizando el factor de descuento:

In [ ]:
print(f'BEL: {(df.salidas_prob @ df.factor_desc):,.2f}')